In [53]:
import os
from datetime import datetime
import pandas as pd
from zeep import Client
import loadenv
import db
from db import get_connection
import psycopg2.extras

In [58]:
def process_county(id):
    cur.execute(f"""
        SELECT timestamp, cases7_per_100k 
        FROM cases_per_county_and_day 
        WHERE county_id = '{id}'
        AND timestamp >= '2021-04-20' ORDER BY timestamp""")
    data = cur.fetchall()
    
    # delete existing ebrake data
    cur.execute(f"DELETE FROM counties_ebrake WHERE id = '{id}'")
    
    retData = []
    for d in data:
        retData.append({
            'id': id,
            'ts': d[0],
            'val': int(d[1]),
            'over100': False,
            'over165': False
        })
            
    inEBrake100 = False
    inEBrake165 = False
    for i in range(4, len(retData)):
                
        # check for date idx = i if ebrake has started
        # must be over t for 3 days
        retData[i]['over100'] = True
        retData[i]['over165'] = True
        for j in range(i - 4, i - 1):
            if data[j][1] < 165:
                retData[i]['over165'] = False
                
            if data[j][1] < 100:
                retData[i]['over100'] = False
                break
                
        if retData[i]['over100'] is True:
            inEBrake100 = True
            
        if retData[i]['over165'] is True:
            inEBrake165 = True
        
        # date is still in eBrake
        if inEBrake100 is True:
            retData[i]['over100'] = True
            
        if inEBrake165 is True:
            retData[i]['over165'] = True
        
        # ebrake can only be terminated on a sunday
        if retData[i]['ts'].isoweekday() == 7:
            over100 = False
            over165 = False
            for j in range(max(0, i - 6), i - 1):
                if retData[j]['val'] >= 100:
                    over100 = True
                    
                if retData[j]['val'] >= 165:
                    over165 = True
                    break
                
            if over165 is False:
                retData[i]['over165'] = False
                inEBrake165 = False
                
            if over100 is False:
                retData[i]['over100'] = False
                inEBrake100 = False
    
    # write to DB:
    psycopg2.extras.execute_values(
        cur, 
        "INSERT INTO counties_ebrake (id, timestamp, ebrake100, ebrake165) VALUES %s", 
        retData,
        template='(%(id)s, %(ts)s, %(over100)s, %(over165)s)',
        page_size=500
    )
    conn.commit()
    
    for d in retData:
        print(d)

In [61]:
conn, cur = get_connection()

cur.execute("SELECT county_id FROM cases_per_county_and_day GROUP BY county_id")
county_ids = cur.fetchall()


for c in county_ids:
    process_county(c[0])
    

{'id': '01001', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 45, 'over100': False, 'over165': False}
{'id': '01001', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 45, 'over100': False, 'over165': False}
{'id': '01001', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 49, 'over100': False, 'over165': False}
{'id': '01001', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': 33, 'over100': False, 'over165': False}
{'id': '01001', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': 42, 'over100': False, 'over165': False}
{'id': '01001', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': 43, 'over100': False, 'over165': False}
{'id': '01002', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 79, 'over100': False, 'over165': False}
{'id': '01002', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 81, 'over100': False, 'over165': False}
{'id': '01002', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 82, 'over100': False, 'over165': False}
{'id': '01002', 'ts': datetime.dateti

{'id': '01062', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 97, 'over100': False, 'over165': False}
{'id': '01062', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 106, 'over100': False, 'over165': False}
{'id': '01062', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 104, 'over100': False, 'over165': False}
{'id': '01062', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': 102, 'over100': False, 'over165': False}
{'id': '01062', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': 104, 'over100': False, 'over165': False}
{'id': '01062', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': 111, 'over100': True, 'over165': False}
{'id': '02000', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 138, 'over100': False, 'over165': False}
{'id': '02000', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 135, 'over100': False, 'over165': False}
{'id': '02000', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 133, 'over100': False, 'over165': False}
{'id': '02000', 'ts': datetime

{'id': '03252', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 109, 'over100': False, 'over165': False}
{'id': '03252', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 121, 'over100': False, 'over165': False}
{'id': '03252', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 137, 'over100': False, 'over165': False}
{'id': '03252', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': 130, 'over100': False, 'over165': False}
{'id': '03252', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': 117, 'over100': True, 'over165': False}
{'id': '03252', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': 105, 'over100': True, 'over165': False}
{'id': '03254', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 101, 'over100': False, 'over165': False}
{'id': '03254', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 110, 'over100': False, 'over165': False}
{'id': '03254', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 112, 'over100': False, 'over165': False}
{'id': '03254', 'ts': datetime

{'id': '03360', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 116, 'over100': False, 'over165': False}
{'id': '03360', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 129, 'over100': False, 'over165': False}
{'id': '03360', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 110, 'over100': False, 'over165': False}
{'id': '03360', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': 100, 'over100': False, 'over165': False}
{'id': '03360', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': 90, 'over100': True, 'over165': False}
{'id': '03360', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': 75, 'over100': True, 'over165': False}
{'id': '03361', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 107, 'over100': False, 'over165': False}
{'id': '03361', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 107, 'over100': False, 'over165': False}
{'id': '03361', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 105, 'over100': False, 'over165': False}
{'id': '03361', 'ts': datetime.d

{'id': '03457', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 138, 'over100': False, 'over165': False}
{'id': '03457', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 126, 'over100': False, 'over165': False}
{'id': '03457', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 106, 'over100': False, 'over165': False}
{'id': '03457', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': 110, 'over100': False, 'over165': False}
{'id': '03457', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': 105, 'over100': True, 'over165': False}
{'id': '03457', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': 101, 'over100': True, 'over165': False}
{'id': '03458', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 129, 'over100': False, 'over165': False}
{'id': '03458', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 122, 'over100': False, 'over165': False}
{'id': '03458', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 129, 'over100': False, 'over165': False}
{'id': '03458', 'ts': datetime

...

{'id': '13071', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 179, 'over100': False, 'over165': False}
{'id': '13071', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 188, 'over100': False, 'over165': False}
{'id': '13071', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 176, 'over100': False, 'over165': False}
{'id': '13071', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': 189, 'over100': False, 'over165': False}
{'id': '13071', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': 178, 'over100': True, 'over165': True}
{'id': '13071', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': 176, 'over100': True, 'over165': True}
{'id': '13072', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 140, 'over100': False, 'over165': False}
{'id': '13072', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 136, 'over100': False, 'over165': False}
{'id': '13072', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 135, 'over100': False, 'over165': False}
{'id': '13072', 'ts': datetime.d

{'id': '14627', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 191, 'over100': False, 'over165': False}
{'id': '14627', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 195, 'over100': False, 'over165': False}
{'id': '14627', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 187, 'over100': False, 'over165': False}
{'id': '14627', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': 185, 'over100': False, 'over165': False}
{'id': '14627', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': 201, 'over100': True, 'over165': True}
{'id': '14627', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': 198, 'over100': True, 'over165': True}
{'id': '14628', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 240, 'over100': False, 'over165': False}
{'id': '14628', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 211, 'over100': False, 'over165': False}
{'id': '14628', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 208, 'over100': False, 'over165': False}
{'id': '14628', 'ts': datetime.d

{'id': '15087', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 211, 'over100': False, 'over165': False}
{'id': '15087', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 195, 'over100': False, 'over165': False}
{'id': '15087', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 204, 'over100': False, 'over165': False}
{'id': '15087', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': 208, 'over100': False, 'over165': False}
{'id': '15087', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': 208, 'over100': True, 'over165': True}
{'id': '15087', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': 194, 'over100': True, 'over165': True}
{'id': '15088', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 256, 'over100': False, 'over165': False}
{'id': '15088', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 221, 'over100': False, 'over165': False}
{'id': '15088', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 207, 'over100': False, 'over165': False}
{'id': '15088', 'ts': datetime.d

{'id': '16064', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 134, 'over100': False, 'over165': False}
{'id': '16064', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 130, 'over100': False, 'over165': False}
{'id': '16064', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 117, 'over100': False, 'over165': False}
{'id': '16064', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': 120, 'over100': False, 'over165': False}
{'id': '16064', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': 132, 'over100': True, 'over165': False}
{'id': '16064', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': 118, 'over100': True, 'over165': False}
{'id': '16065', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 211, 'over100': False, 'over165': False}
{'id': '16065', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 216, 'over100': False, 'over165': False}
{'id': '16065', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 220, 'over100': False, 'over165': False}
{'id': '16065', 'ts': datetime

{'id': '16077', 'ts': datetime.datetime(2021, 4, 20, 0, 0), 'val': 205, 'over100': False, 'over165': False}
{'id': '16077', 'ts': datetime.datetime(2021, 4, 21, 0, 0), 'val': 199, 'over100': False, 'over165': False}
{'id': '16077', 'ts': datetime.datetime(2021, 4, 22, 0, 0), 'val': 193, 'over100': False, 'over165': False}
{'id': '16077', 'ts': datetime.datetime(2021, 4, 23, 0, 0), 'val': 243, 'over100': False, 'over165': False}
{'id': '16077', 'ts': datetime.datetime(2021, 4, 24, 0, 0), 'val': 229, 'over100': True, 'over165': True}
{'id': '16077', 'ts': datetime.datetime(2021, 4, 25, 0, 0), 'val': 229, 'over100': True, 'over165': True}
